<a href="https://colab.research.google.com/github/arcadyb/pyreport/blob/main/mergetopdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install Pillow reportlab
from reportlab.pdfbase.pdfmetrics import stringWidth


In [9]:
from google.colab import drive
drive.mount('/content/drive')
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase.pdfmetrics import stringWidth

import os
import re
import tempfile
from reportlab.pdfbase.pdfmetrics import stringWidth

# Assuming rotate_image and extract_info_from_filename functions are defined here
def rotate_image(img, angle):
    return img.rotate(angle, expand=True)
def extract_info_from_filename(filename):
    pattern = r'order-(?P<order>\d+)_rot-(?P<rot>-?\d+)_title-(?P<title>[^_]+)_date-\((?P<date>.+)\)'
    match = re.search(pattern, filename)
    if match:
        return match.groupdict()
    return None

folder_path = '/content/drive/My Drive/ColabWSData/reports/makolet1'
output_pdf = '/content/drive/My Drive/ColabWSData/reports/kabalot_no_toc.pdf'

image_files_info = []
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        info = extract_info_from_filename(filename)
        if info:
            info['filename'] = filename
            image_files_info.append(info)

image_files_info.sort(key=lambda x: int(x['order']))

c = canvas.Canvas(output_pdf)
title_height = 140  # Adjusted height of the title area for larger text

for info in image_files_info:
    image_path = os.path.join(folder_path, info['filename'])
    with Image.open(image_path) as img:
        rotated_img = rotate_image(img, int(info['rot']))
        width, height = rotated_img.size
        # Adjust page size for title
        c.setPageSize((width, height + title_height))

        # Convert image to a temporary file
        with tempfile.NamedTemporaryFile(delete=True, suffix='.png') as tmp:
            rotated_img.save(tmp.name, format='PNG')
            # Draw the image lower to make room for the title
            c.drawImage(tmp.name, 0, title_height, width=width, height=height)

            # Draw a contrasting background for the title to ensure visibility
            c.setFillColorRGB(1, 1, 1)  # White background for the title area
            c.rect(0, height + title_height - 60, width, 60, fill=1)

            # Add the title text in a contrasting color, centered and twice as large
            font_size = 24  # Double the original size
            c.setFont("Helvetica", font_size)
            text = f"{info['date']} - {info['title']}"
            text_width = stringWidth(text, "Helvetica", font_size)
            c.setFillColorRGB(0, 0, 0)  # Black text
            # Calculate the x position for centered text
            text_x_position = (width - text_width) / 2
            c.drawString(text_x_position, height + title_height - 40, text)

            c.showPage()

c.save()
print(f'PDF created with {len(image_files_info)} images.')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PDF created with 2 images.
